### CppCon 2017: Fedor Pikus “C++ atomics, from basic to advanced. What do they really do?”

https://www.youtube.com/watch?v=ZQFzMfHIxng&list=PLR2BwNxHx0z8ccXrXKzuTnsfB17fidY3M&t=2962s&index=10

#### 2:25 https://youtu.be/ZQFzMfHIxng?t=145

In [1]:
#include <atomic>
#include <cstddef>
#include <iostream>
#include <mutex>

In [2]:
// Program A:
{
    const size_t N = 5;
    long a[N] = { 1, 2, 3, 4, 5 }; // can be very long

    std::atomic<long> sum{};
    auto do_work = [&](size_t N, long* a) {
        for (size_t i = 0; i < N; ++i) {
            sum += a[i];
        }
    };

    do_work(N, a);
    std::cout << sum << '\n';
}

15


In [3]:
// Program B:
{
    const size_t N = 5;
    long a[N] = { 1, 2, 3, 4, 5 }; // can be very long

    long sum(0);
    std::mutex M;
    auto do_work = [&](size_t N, long* a) {
        long s = 0;
        for (size_t i = 0; i < N; ++i) {
            s += a[i];
        }
        std::lock_guard<std::mutex> L(M);
        sum += s;
    };

    do_work(N, a);
    std::cout << sum << '\n';
}

15


**2:32 and 3:07** shows "lock free" 要慢很多！why?

### Is lock-free faster?
- Algorithm rules supreme **算法最关键**
- "Wait-free" has nothing to do with time
  - Wait-free refers to the number of compute "steps"
  - Steps do not have to be of the same duration
- **Atomic operations do not guarantee good performance**
- There is no substitute for understanding what you're doing
  - This class is the next best thing

#### 4:24 What is an atomic operation?
#### 8:50 Date shareing in C++

In [4]:
{
    std::atomic<int> x(0); // good
    // std::atomic<int> x = 0; // bad - but compiles! cling bug?

    ++x; // atomic!!!
    std::cout << x;
}


1

#### 10:44 What types can be made atomic?
- Any **trivially copyable** type can be made atomic
- What is trivially copyable?
  - Continues chunk of memory
  - Copying the object means copying all bits (memcpy)
  - No virtual functions, noexcept constructor

#### 11:55 What operations can be done on `std::atomic<int>`?

One of these is not the same as the others:
```cpp
++x;
x++;
x += 1;
x != 2;
x *= 2;
int y = x * 2;
x = y + 1;
x = x + 1;
x = x * 2;
```

- `x *= 2;` // this does not compile! There is no automic multiply in most hardware.只要能编译，C++需要保证该操作是atomic。
- 最后两个也不是atomic，尽管它们能编译
  - 一行里面有两个原子操作！Atomic read followed by atomic write!
  - 另外一个线程可以在这两个原子操作之间改变这个原子变量


#### `std::atomic<T>` and overloaded operators

- `std::atomic<T>`只对可以原子运算的操作提供重载，否则不编译
- 注意：包含原子变量的表达式还是可以编译的，问题在于整个表达式未必是一个原子操作。**这非常容易导致错误！**

#### 15:41 What "other operations" can be done on `std::atomic<T>`?

- Explicit reads/writes
```cpp
T y = x.load(); // same as T y = x;
x.store(y); // same as x = y;
```
- Atomic exchange:
```cpp
T z = x.exchange(y); // Atomically: z = x; x = y;
```

- Compare-and-swap (conditional exchange):
```cpp
bool success = x.compare_exchange_strong(y, z); // T& y;
    // if x==y, make x=z and return true;
    // Otherwise, set y=x and return false
```

- CAS is the **key to most lock-free algorithms**

#### 17:07 What is so special about CAS?

##### Example: atomic increment with CAS:

In [5]:
{
    std::atomic<int> x{0};
    int x0 = x;
    while(!x.compare_exchange_strong(x0, x0+1)) {}
    
    std::cout << "x = " << x << ", x0 = " << x0;
}

x = 1, x0 = 0

上面例子里：
- 如果没有其它线程在操作这个原子变量x，直接就能成功，返回true，循环退出
- 如果有其它线程也有CAS在操作x，**x0会再一次赋值成改变后的x**，返回false，让循环重复，知道这个CAS beats 其它人的CAS
- 这里是**lock-free，不是wait-free**

某些lock-free更简单，但是这个能做任何事情：
- increment doubles
- multiply integers
- and may more
```cpp
while(!x.compare_exchange_strong(x0, x0*2)) {}
```

#### 18:54 What "other operations" can be done on `std::atomic<T>`?

- fetch_add

In [6]:
{
    std::atomic<int> x{1};
    int y = 2;
    int z = x.fetch_add(y); // same as x += y, but return old x
    std::cout << "x = " << x << ", z = " << z;
}

x = 3, z = 1

- fetch_sub, fetch_and, fetch_or, fetch_xor
  - same as +=, -=, etc. operators
  
- more verbose, but **less error-prone than operators and expressions**
  - 原因是operators and expressions不容易发现整个表达式是由多个atomic操作组成的，但整体并不是atomic
  - 但是如果有多个这些function calls，更容易让人理解成对应于单个的atomic操作，直觉上并不觉得组合起来是atomic

#### 21:22 How fast are atomic operations?
- Performance should be measured
- 硬件相关！编译器相关！

#### 22:30 atomic vs. non-atomic 比较结果
- atomic 略微慢一点

#### 23:19 atomic vs. locks 比较结果
- mutex 要慢不少
- spinlock几乎和atomic差不多
- 26:23: CAS比atomic/spinlock慢一些，但是比mutex快，介于两者之间

#### 26:30 Is  atomic the same as lock-free?
std::atomic隐藏了一个天大的秘密，并不总是lock-free

In [7]:
{
    struct A {long x;};
    struct B {long x; long y;};
    struct C {long x; long y; long z;};
    std::cout << "A lock-free: " << std::atomic<A>{}.is_lock_free() << '\n'
              << "B lock-free: " << std::atomic<B>{}.is_lock_free() << '\n'; // maybe
            //<< "C lock-free: " << std::atomic<C>{}.is_lock_free() << '\n'; // cling error, should return 0
}

A lock-free: 1
B lock-free: 1


- is_lock_free() 是runtime function，为什么不是compile time?
  - 原因是alignment
- c++17提供了一个compile time function:
  - constexpr is_always_lock_free()

#### 29:43 Do atomic operations wait on each other?
Testing of 3 cases:
1. shared: `std::atomic<int> x;` ++x in two threads
2. not shared: like above one, but in different cachelines
3. non-shared (false sharing): `std::atomic<int> x[2];` ++x[0] in thread1, ++x[1] in thread2
  - this is actually falsed shared, because x[0] and x[1] are in the same cacheline

The testing result is at 31:52
- case 1 and 3 is worse than 2

**结论**：
-  原子操作确实要互相等待，需要等待cache line的访问
  - 这是date sharing without races要付出的代价
  - 即使对不同的原子变量访问，也可能会落到相同的 cache line 上（false sharing），仍然付出real-time penalty

#### 33.03 Strong and weak CAS

- x.compare_exchange_strong(old_x, new_x); // T& old_x
```cpp
if (x == old_x) { x = new_x; return true; }
else { old = x; return false; }
```
- x.compare_exchange_weak(old_x, new_x);
  - same thing, but can "spuriously fail" and return false even if x == old_x
  - what is the value of old_x if this happens?
  - if weak CAS correctly returns x == old_x, why would it fail?

compare_exchange_weak对x86系统，描述的问题不会发生，但是对其它不同的硬件，可能会发生

##### CAS, concepturally (pseudo-code):
```cpp
bool compare_exchange_strong(T& old_v, T new_v) {
    Lock L;        // Get exclusive access
    T tmp = value; // Current value of the atomic
    if (tmp != old_v) { 
        old_v = tmp;
        return false;
    }
    value = new_v;
    return true;
}
```
Lock不是真的mutex，而是硬件实现的某些排他性访问机制

##### 35:39 Read is faster than write:
```cpp
bool compare_exchange_strong(T& old_v, T new_v) {
    T tmp = value;  // Current value of the atomic
    if (tmp != old_v) {
        old_v = tmp;
        return false;
    }
    Lock L;         // Get exclusive access
    tmp = value;    // value could have changed!
    if (tmp != old_v) {
        old_v = tmp;
        return false;
    }
    value = new_v;
    return true;
}
```
Double-checked locking pattern is back! 不过这是硬件来实现的

##### 36:27 If exclusive access is hard to get, let someone else try:
```cpp
bool compare_exchange_weak(T& old_v, T new_v) {
    T tmp = value;  // Current value of the atomic
    if (tmp != old_v) {
        old_v = tmp;
        return false;
    }
    TimedLock L;    // Get exclusive access
    if (!L.locked()) return false;  // old_v is correct
    tmp = value;    // value could have changed!
    if (tmp != old_v) {
        old_v = tmp;
        return false;
    }
    value = new_v;
    return true;
}
```
Exclusive access在某些硬件平台上很难获得（X86没有这个问题）。Read可以，但是Exclusive access不容易。**所以 Lock => TimedLock**

#### 37:37 There is MUCH more ...
很少有人直接使用atomic变量，一般都是配合non-atomic变量一起使用

- Atomic Queue

```cpp
int q[N];
std::atomic<size_t> front;
void push(int x) {
    size_t my_slot = front.fetch_add(1);
    q[my_slot] = x;
}
```

- 这是一个lock-free queue，用atomic来实现的。
- 这里的front是atomic类型，指向front的index。
- 先不考虑多个线程write的情况，关于reader后面再说
- my_slot如果为0，front为1，表示`q[0]`归当前的些操作

**关键：**atomic变量作为非atomic数据的index，几乎所有lock-free的数据结构都是类似的方法。不一定是作为index，也可以作为指针。

#### 39:48 Atomic list

```cpp
struct node {
    int value;
    node* next;
};
std::atomic<node*> head; // atomic用作指针指向non-atomic

void push_front(int x) {
    node *new_n = new node;
    new_n->value = x;
    node* old_h = head;
    do {
        new_n->next = old_h;
    } while(!head.compare_exchange_strong(old_h, new_n)); // new_n: new node is new head
                                                          // old_h: head has not changed
}
```

**关键：**atomic变量作为可以作为指针，指向non-atomic的数据。几乎所有lock-free算法都和这类似

#### 41:23 Atomic variables as gateways to memory access (generalized pointers)

- 利用原子变量得到对普通内存(non-atomic)的独占访问
- 怎么保证其它线程看到的普通内存是需要的final状态，而不是中间的混乱状态呢？
  - memory barriers!

#### 42:15 Memory barriers - the other side of atomics

- Memory barriers控制一个CPU对内存的操作，怎么对另外一个CPU变得可见。
- 如果没有Memory barriers，或者对应的其它东西，CPU们各自操作它们的cache，而主存根本就不必（及时的）进行相应的变化。不能保证任何人看到任何东西。
- X86其实不是这样，但是有些系统是这样的。





#### 42:58 Memory barriers

- 是全局的
- 控制多CPU的数据可见性
- 必须由硬件控制
- 一般不是由特殊指令实现的（尽管有这个可能）
  - 一般是其它指令的attributes，用来修饰其它指令

#### 43:13 Memory barriers in C++

- C++11 引入memory barriers
- memory barriers 和 memoyr order 关系紧密。memory barriers 是 memory order 背后的支撑


In [8]:
{
    std::atomic<int> x;
    x.store(1, std::memory_order_release); // put a release-memory-barrier on the store
}

#### 44:08 No memory barrier: std::memory_order_relaxed
- 意味着：可以随意reorder reads/writes
- 下面的例子，
  - a, b, c, x的write，在代码上是顺序发生的，
  - 但实际顺序可以是乱的，任何顺序都是可能发生的

```cpp
int a, b, c;
std::atomic<int> x;
a = 1;
b = 2;
c = 3;
x.fetch_add(1, std::memory_order_relaxed);
```

#### 44:40 Acquire barrier

- Acquire barrier，也叫half barrier
- **保证**：所有的程序代码上放在barrier之后操作，只在此barrier之后可见
  - 所有的操作，包括读和写，而不是只有读，或者只有写
  - 所有的操作，不光光是原子变量的，也包含其它变量的
- 程序代码中barrier之前的操作（读和写），可能被重排序到barrier之后
- 而代码上barrier之后的操作，不会被重拍到barrier之前

#### 45:48 Release barrier

- **保证**：代码上barrier之前的操作，在barrier之前可见
- 读写操作不会被重排序到barrier之后
- 例如：代码顺序上barrier之后的某个store操作可能被在barrier之前观察到，但代码上barrier之前的store不会在barrier之后可见

#### 46:10 Acquire-release order

- Acuqire and release barriers通常成对出现
  - thread 1 writes 原子变量 **x**，使用release barrier，能保证所有在代码顺序上barrier之前的写操作可见
  - thread 2 reads 原子变量 **x**，使用 acquire barrier，保证代码顺序上barrier之后的读操作确实是在barrier之后执行的
- 这样，barrier之前的写确实写了，而读保证发生在barrier之后
- 必须是对同一个原子变量操作
- 之所以叫做release，thread 1有些private data，一番操作后，可以publish了，so release it
- Thread 2, 使用acquire barrier，需要和thread 1的数据同步，在barrier之后的数据可见，保证acquire the data published in thread 1

#### 48:12 Barriers and locks

Acquire and release barriers are used in locks
```cpp
Lock L;        std::atomic<int> l(0);
L.lock();      l.store(1, std::memory_order_acquire);
++x;           ++x;
L.unlock();    l.store(0, std::morory_order_release);
```

```cpp
Lock L;        std::atomic<int> l(0);
L.lock();      while (l.exchange(1, std::memory_order_qcquire));
++x;           ++x;
L.unlock();    l.store(0, std::morory_order_release);
```

视频中没有详细讲其中的原理。

#### 48:18 Bidirectional barriers

- Acquire-Release (std::**memory_order_acq_rel**)
  - combines acquire and release barriers，没有操作能跨越这个barrier
  - 两个线程必须使用相同的原子变量，只有这样才能工作
  
- Sequence consistency (std::**memory_order_seq_cst**)
  - 最严格的barrier
  - 不再要求使用相同的原子变量
  - 假设对两个不同的原子变量使用这个barrier，你也能保证得到正确的顺序

#### 48:50 为什么CAS有两个memory orders?

- CAS是仅有的带有两个memory order参数的函数
- 一个用作read，一个用作write

```cpp
bool compare_exchange_strong(T& old_v, T new_v, momory_order on_success, momory_order on_failure) {
    T tmp = value.load(on_failure);
    if (tmp != old_v) {
        old_v = tmp;
        return false;
    }
    Lock L;         // Get exclusive access
    tmp = value;    // value could have changed!
    if (tmp != old_v) {
        old_v = tmp;
        return false;
    }
    value.store(new_v, on_success);
    return true;
}
```

#### 49:27 Default memory order
- std::memory_order_seq_cst - the strongest order
- but really expensive

**50:06** seq_cst的写，比起relaxed的写，以及non-atomic的写要**慢1.5个数量级**。后面两者非常接近

#### 50:49 Memory barriers are expensive

- Memory barriers可能比原子操作本身更expensive
- 具体和硬件相关。对于ARM，非常昂贵
- On X86:
  - **all loads are acquire loads, all stores are release stores。这两种情况在X86上是免费的**
  - 但是其它的 memory barriers 很昂贵，例如 release on read, acquire on write
  - 所有的读改写（read-modify-write），例如 exchange，都有 bidirectional barriers
  - **acq-rel 和 seq_cst对于x86是相同的**


#### 51:32 Memory order expresses programmer's intent

- lock-free代码很难写
- 也很难读，别人难读，自己也难读
- Memory order specification可以表达你程序员到底想做什么

```cpp
std::atomic<size_t> count;
count.fetch_add(1, std::memory_order_relaxed);
```

- 对于上面的代码，要表达的是：
  - count 被并发的自增，并不被用作其它数据的 index，也不被用作 reference count(没有其它的内存访问依赖于此)
  - 读者怎么知道的？因为没有用任何 memory barrier。（relaxed 就是没有 barrier）
  - 就是一种 counter
- **NOTE：**在x86上，fetch_add实际上是 memory_order_acq_rel，你可能想要偷工减料，但不要这样做：
  - 这样可能让别人，甚至自己confused
  - 编译器也很有可能利用此信息来优化重排序（尽管现在并没有，尽管很难，但不是没有可能）

#### 53:09 Memory order expresses programmer's intent - 另一个例子

```cpp
std::atomic<size_t> count;
count.fetch_add(1, std::memory_order_release);
```

- 对于上面的代码，要表达的是：
  - count作为其它数据的index。当其它数据在此线程准备好之后，可以被release了
  - 代码如下：
```cpp
T data[max_count];
initialize(data[count.load(std::memory_order_relaxed)]); // nobody can see new data yet
count.fetch_add(1, std::memory_order_release); // now they can see it
```
为了能正确的使用这个data[]，我必须使用acquire barrier来load

#### 53:51 Memory order expresses programmer's intent - 另一个例子

```c++
std::atomic<size_t> count;
++count;
```

两种可能性：
1. 有超过一个原子变量，所以需要sequential consistency (default memory order) 来同步它们
- 更有可能，你并没有仔细考虑你在做什么。也许你遇到了bug，只好乱改改 memory order，直到bug消失

#### 55:02 关于 sequential consistency的要点

- Sequential consistency 让程序可读，并且一般并没有performance penalty
- 但没有必要让每个原子操作都使用 memory_order_seq_cst，这样会丢失了程序员的意图

#### 55:37 关于C++标准库的抱怨

假设有如下代码：
```cppp
class C {
    std::atomic<size_t> N;
    T* p;
    ...
};

C::~C() {
    cleanup(p, N.load(std::memory_order_relaxed));
}
```

- 这里使用原子方式的load，隐含表达了其它线程在对象销毁的时候，竟然还在访问此对象！
- 当然，有可能是C++标准并没有提供 **N.load_nonatomic()**，这里不得不吓唬读者了

#### 56:46 C++ and std::atomic